In [2]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np
import winsound
import requests

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import datetime
year = datetime.date.today().year

In [3]:
#webdriver.__version__
import selenium
selenium.__version__
#import selenium

'4.15.2'

In [33]:
service = Service(executable_path = 'C:\Python\chromedriver.exe')
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("start_maximized")

driver = webdriver.Chrome(service = service, options = options)

stealth(driver,
#        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
        languages=["en-US", "en"],
        vendor = "Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=False, #True 
        run_on_insecure_origins=False
        )

link_all = []

url = f"https://dom.gosuslugi.ru/#!/houses"

driver.get(url)
sleep(3)

In [34]:
driver.find_element(By.XPATH, '//*[@id="s2id_regionpa1"]/a').click() #находим выпадающий список Субъект РФ
input_1 = driver.find_element(By.XPATH, '//*[@id="s2id_autogen6_search"]') #находим input Субъект РФ
input_1.send_keys('Приморский край') #вносим в ячейку Приморский край
sleep(2) #спим 2 секунды
input_1.send_keys(Keys.ENTER) #жмем Enter

driver.find_element(By.XPATH, '//*[@id="s2id_citypa1"]/a').click() #находим выпадающий список Город
input_2 = driver.find_element(By.XPATH, '//*[@id="s2id_autogen10_search"]') #находим input
input_2.send_keys('Владивосток') #вносим в ячейку г. Владивосток
sleep(2) #спим 2 секунды
input_2.send_keys(Keys.ENTER) #жмем Enter

driver.find_element(By.XPATH, '//*[@id="s2id_autogen25"]').click() #находим выпадающий список Тип дома
sleep(2)
driver.find_element(By.XPATH, '//*[@id="select2-drop"]/ul/li[1]').click() #Выбираем Многоквартирный или Жилой
sleep(3)
# //*[@id="select2-drop"]/ul/li[1] #Жилое
# //*[@id="select2-drop"]/ul/li[3] #Многоквартирный

#button = driver.find_element(By.XPATH, '/html/body/div[1]/div[8]/div/div[2]/houses-search-form/div[2]/div/ef-bp-form/div/form/div[3]/div/div[2]/button') #находим элемент кнопки Еще
#button.send_keys(Keys.ENTER) #нажимаем кнопку Найти
#sleep(10) #спим 10 секунд

In [365]:
#Парсим все улицы
soup = BeautifulSoup(driver.page_source, 'lxml')

street = soup.findAll('div', class_ = 'select2-result-label')

streets = []

for st in street:
    streets.append(st.text)

streets
#len(table_2)

['пр-кт. 100-летия Владивостока',
 'ул. 10-я Рабочая',
 'ул. 13-я Рабочая',
 'ул. 1-я Восточная',
 'ул. 1-я Встречная',
 'ул. 1-я Гражданская',
 'ул. 1-я Дачная',
 'ул. 1-я Заречная',
 'ул. 1-я Круговая',
 'ул. 1-я Крутая',
 'ул. 1-я Лесная',
 'ул. 1-я Линейная',
 'ул. 1-я Матросская',
 'ул. 1-я Морская',
 'ул. 1-я Поселковая',
 'ул. 1-я Пригородная',
 'ул. 1-я Промышленная',
 'ул. 1-я Проходная',
 'ул. 1-я Северная',
 'ул. 1-я Флотская',
 'ул. 1-я Центральная',
 'ул. 1-я Шоссейная',
 'ул. 20 лет ВЛКСМ',
 'ул. 2-я Водопроводная',
 'ул. 2-я Восточная',
 'ул. 2-я Гражданская',
 'ул. 2-я Круговая',
 'ул. 2-я Крутая',
 'ул. 2-я Лесная',
 'ул. 2-я Линейная',
 'ул. 2-я Матросская',
 'ул. 2-я Поселковая',
 'ул. 2-я Пригородная',
 'ул. 2-я Промышленная',
 'ул. 2-я Проходная',
 'ул. 2-я Строительная',
 'ул. 2-я Центральная',
 'ул. 2-я Шоссейная',
 'ул. 2-я Южная',
 'ул. 3-ий Ключ',
 'пер. 3-й Восточный',
 'ул. 3-я Восточная',
 'ул. 3-я Встречная',
 'ул. 3-я Гарнизонная',
 'ул. 3-я Круговая',
 '

In [366]:
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_streets.xlsx')
pd.DataFrame(streets).to_excel(writer, 'street')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\692976354.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [373]:
#Сплитим улицы
street_1 = []

for i in range(0, len(streets), 1):
    stree = streets[i].split('.')[1].strip()
    street_1.append(stree)

In [374]:
street_1

['100-летия Владивостока',
 '10-я Рабочая',
 '13-я Рабочая',
 '1-я Восточная',
 '1-я Встречная',
 '1-я Гражданская',
 '1-я Дачная',
 '1-я Заречная',
 '1-я Круговая',
 '1-я Крутая',
 '1-я Лесная',
 '1-я Линейная',
 '1-я Матросская',
 '1-я Морская',
 '1-я Поселковая',
 '1-я Пригородная',
 '1-я Промышленная',
 '1-я Проходная',
 '1-я Северная',
 '1-я Флотская',
 '1-я Центральная',
 '1-я Шоссейная',
 '20 лет ВЛКСМ',
 '2-я Водопроводная',
 '2-я Восточная',
 '2-я Гражданская',
 '2-я Круговая',
 '2-я Крутая',
 '2-я Лесная',
 '2-я Линейная',
 '2-я Матросская',
 '2-я Поселковая',
 '2-я Пригородная',
 '2-я Промышленная',
 '2-я Проходная',
 '2-я Строительная',
 '2-я Центральная',
 '2-я Шоссейная',
 '2-я Южная',
 '3-ий Ключ',
 '3-й Восточный',
 '3-я Восточная',
 '3-я Встречная',
 '3-я Гарнизонная',
 '3-я Круговая',
 '3-я Крутая',
 '3-я Лесная',
 '3-я Линейная',
 '3-я Матросская',
 '3-я Поселковая',
 '3-я Пригородная',
 '3-я Проходная',
 '3-я Северная',
 '3-я Строительная',
 '3-я Центральная',
 '3-я

In [375]:
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_streets_2.xlsx')
pd.DataFrame(street_1).to_excel(writer, 'street')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\3177908438.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [6]:
#Подгружаем улицы
df_str = pd.read_excel('D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Квартиры_Владивосток\ГИС ЖКХ/База_ГИС_ЖКХ_streets.xlsx', sheet_name = 1)
df_str

,Улицы_yes_ж
0,10-я Рабочая
1,1-я Восточная
2,1-я Встречная
3,1-я Гражданская
4,1-я Дачная
...,...
437,Янковского
438,Ярославская
439,Ясеневая
440,Ясный


In [7]:
len(df_str['Улицы_yes_ж'])

442

In [448]:
#Парсинг через улицы
#ВАРИАНТ №1 - не сработал
a = 0
street_no = [] #улицы которые не прошли и они пустые
street_big_100 = [] #улицы которые больше 100 домов
street_all_count = [] #количество домов по каждой улице
street_yes_no = [] #улицы которые не прошли и они не пустые

for stre in tqdm(range(0, len(df_str['Улицы_2']), 1), 'Сбор данных'):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    a = a + 1
    try:
        #print(df['Улицы_2'][stre])
        try:
            driver.find_element(By.XPATH, '//*[@id="s2id_streetpa1"]/a').click() #находим выпадающий список Улица
            input_3 = driver.find_element(By.XPATH, '//*[@id="s2id_autogen16_search"]') #находим input
            input_3.send_keys(df_str['Улицы_2'][stre]) #вносим в ячейку улицу
            sleep(2) #спим 2 секунды
            input_3.send_keys(Keys.ENTER) #жмем Enter
        except:
            #input_3.send_keys(Keys.ENTER) #жмем Enter
            winsound.Beep(2000, 3000) #громкий звук в 3 секунд
            sleep(60)       

        button = driver.find_element(By.XPATH, '/html/body/div[1]/div[8]/div/div[2]/houses-search-form/div[2]/div/ef-bp-form/div/form/div[3]/div/div[2]/button') #находим элемент кнопки Еще
        button.send_keys(Keys.ENTER) #нажимаем кнопку Найти
        sleep(6) #спим 6 секунд
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        #СОХРАНИМ УЛИЦЫ КОТОРЫХ БОЛЬШЕ 100
        try:
            N = int(soup.find('b', class_ = 'ng-binding').text) #количество объектов
            street_all_count.append(N)
            if N > 100:
                street_big_100.append(df_str['Улицы_2'][stre])
        except:
            continue
        
        #АДРЕС
        address = driver.find_elements(By.CLASS_NAME, "register-card__header-title")

        #СОСТОЯНИЕ
        sost = soup.findAll('td', class_ = 'col-xs-6')

        sosts = []
        sosts_2 = []

        #for i, s in enumerate(sost):
        for s in sost:
            try:
                sosts.append(s.find('span', class_ = 'ng-binding').text.strip())
            except:
                continue

        i = 0
        for s in sosts:
            if i % 3 == 0:
                sosts_2.append(s)
            i = i + 1
        #sosts_2

        #КН, ГОД, МАТЕРИАЛ, ЭТАЖНОСТЬ + АДРЕС и СОСТОЯНИЕ
        table_2 = soup.findAll('td', class_ = 'col-xs-7')

        table_s_2 = []

        i = 0
        j = 0
        for tb in table_2:
            table_s_2.append(tb.text)
            i = i + 1
            if i % 4 == 0:
                j = j + 1
                #print(j, i, address[j - 1].text)
                table_s_2.append(address[j - 1].text)
                table_s_2.append(sosts_2[j - 1])
        #table_s_2
        
        #СОБИРАЕМ В ТАБЛИЦУ
        new_array = np.reshape(table_s_2, (N, 6))
        df = pd.DataFrame(new_array, columns = ['КН', 'Год', 'Материал', 'Этажность', 'Адрес', 'Cостояние'])

        if a == 1:
            df_result = df
        if a > 1:
            df_result = pd.concat([df_result, df], axis = 0)
    
    except:
        #print('Error')
        street_no.append(df_str['Улицы_2'][stre])
        try:
            N = int(soup.find('b', class_ = 'ng-binding').text)
            if N > 0:
                street_yes_no.append(df_str['Улицы_2'][stre])
                #winsound.Beep(2000, 3000) #громкий звук в 3 секунд
                #sleep(60)
        except:
            continue
        
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные_all.xlsx')
df_result.to_excel(writer, 'База')
writer.save()

Сбор данных: 100%|███████████████████████████████████████████████████████████████| 1001/1001 [2:23:04<00:00,  8.58s/it]
C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\2986323560.py:100: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [35]:
#Новый парсинг через таблички по улицам
#ВАРИАНТ №2 - сработал, итоговый
#Необходимо обязательно выставить 100 объектов на странице
#Парсить надо только те улицы, по которым есть объекты. Перед парсингом необходимо собрать улицы с объектами

street_no = [] #улицы которые не прошли и они пустые
street_big_100 = [] #улицы которые больше 100 домов
street_all_count = [] #количество домов по каждой улице
objeckt = []
#street_yes_no = [] #улицы которые не прошли и они не пустые
a = 0

for stre in tqdm(range(0, len(df_str['Улицы_yes_ж']), 1), 'Сбор данных'):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    try:
        try:
            driver.find_element(By.XPATH, '//*[@id="s2id_streetpa1"]/a').click() #находим выпадающий список Улица
            input_3 = driver.find_element(By.XPATH, '//*[@id="s2id_autogen16_search"]') #находим input
            input_3.send_keys(df_str['Улицы_yes_ж'][stre]) #вносим в ячейку улицу
            sleep(2) #спим 2 секунды
            input_3.send_keys(Keys.ENTER) #жмем Enter
        except:
            #input_3.send_keys(Keys.ENTER) #жмем Enter
            winsound.Beep(2000, 3000) #громкий звук в 3 секунд
            sleep(30)       

        button = driver.find_element(By.XPATH, '/html/body/div[1]/div[8]/div/div[2]/houses-search-form/div[2]/div/ef-bp-form/div/form/div[3]/div/div[2]/button') #находим элемент кнопки Еще
        button.send_keys(Keys.ENTER) #нажимаем кнопку Найти
        sleep(6) #спим 6 секунд
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        #СОХРАНИМ УЛИЦЫ КОТОРЫХ БОЛЬШЕ 100
        N = int(soup.find('b', class_ = 'ng-binding').text) #количество объектов
        street_all_count.append(N)
#         try:
#             obj = driver.find_element(By.XPATH, '//*[@id="select2-chosen-201"]').text
#             objeckt.append(obj)
            
#             N = int(soup.find('b', class_ = 'ng-binding').text) #количество объектов
#             #street_all_count.append(N)
#             if N > 100:
#                 street_big_100.append(df_str['Улицы_yes_ж'][stre])
#         except:
#             continue

        t1 = []
        t = []
        
        #СРБИРАЕМ ТАБЛИЧКИ
        table = soup.findAll('div', class_ = 'register-card__container_multi')  #[2].findAll('td')[10].text.strip()
        address = soup.findAll('div', class_ = 'register-card__header-title')   #адрес
        err = soup.findAll('div', class_ = 'register-card__container_multi')[0] #проверка на пустой список, если пусто,
                                                                                #то будет ошибка и вылет в except
        for tb, ad in zip(table, address):                          #перебираем таблицы и адреса (они равны по количеству)
            t1.clear()                                              #очищаем список t1
            t.clear()                                               #очищаем список t
            a = a + 1                                               #счетчик

            value = tb.findAll('td')                                #вытаскиваем все тэги td

            for vl in value:
                t.append(vl.text.strip())                           #создаем текстовый список

            splits = np.array_split(t, len(value) / 2)              #разбиваем список на пары
            for array in splits:
                t1.append(list(array))                              #все пары переносим в список t1
            
            t1.append(['Адрес:', ad.text.strip()])                  #добавляем адрес к списку t1, таблица собрана

            df = pd.DataFrame(t1)                                   #переводим таблицу в df
            df = df.set_index(0)                                    #первый столбик делаем индексом для конката  

            if a == 1:                                              #в ппервый проход создаем df
                df_result = df
            if a > 1:
                df_result = pd.concat([df_result, df], axis = 1)    #потом добавляем таблички к результату
            #print(len(df_result))
    except:
        #print(N)
        street_no.append(df_str['Улицы_yes_ж'][stre])               #улицы которые не прошли, попадают в список

df_result_copy = df_result.copy()                                   #делаем копию
df_result_copy = df_result_copy.transpose()                         #транспонируем, строки становятся столбцами

#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные_жилое.xlsx')
df_result_copy.to_excel(writer, 'База')
writer.save()

Сбор данных: 100%|█████████████████████████████████████████████████████████████████| 442/442 [1:04:21<00:00,  8.74s/it]
C:\Users\skapr\AppData\Local\Temp\ipykernel_6628\985473606.py:89: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [37]:
df_result_copy

,Кадастровый номер:,Год ввода в эксплуатацию:,Количество этажей:,Дом находится всобственности субъектаРФ и…\n \n\n\n Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится вмуниципальнойсобственности и…\n \n\n\n Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Состояние:,Управляющая организация:,Адрес:
1,25:28:010031:300,-,1,Нет,Нет,Исправный,-,"690066, край Приморский, г Владивосток, ул 10-..."
1,25:28:000000:17704,-,1,Нет,Нет,Исправный,-,"690066, край Приморский, г Владивосток, ул 10-..."
1,25:28:050022:695,1956,1,NaN,NaN,-,-,"690054, край Приморский, г Владивосток, ул 1-я..."
1,25:28:050022:735,-,1,NaN,Нет,Исправный,-,"690054, край Приморский, г Владивосток, ул 1-я..."
1,25:28:050023:901,-,1,Нет,Нет,-,-,"690054, край Приморский, г Владивосток, ул 1-я..."
...,...,...,...,...,...,...,...,...
1,25:28:030002:473,1958,1,NaN,NaN,-,-,"690012, край Приморский, г Владивосток, ул Ясе..."
1,25:28:030002:332,1958,2,NaN,NaN,-,-,"690012, край Приморский, г Владивосток, ул Ясе..."
1,25:28:030002:393,1958,1,NaN,NaN,-,-,"690012, край Приморский, г Владивосток, ул Ясе..."
1,25:28:000000:879,-,2,NaN,Нет,-,-,"690041, край Приморский, г Владивосток, пер Яс..."


In [103]:
np.sum(street_all_count)

3736

In [104]:
len(street_all_count)

330

In [63]:
#Сохраняем количество объекто по улицам
writer = pd.ExcelWriter('База_ГИС_ЖКХ_количество.xlsx')
pd.DataFrame(street_all_count).to_excel(writer, 'количество')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_10060\1899113572.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [58]:
street_big_100

['100-летия Владивостока', 'Восток', 'Русская']

In [61]:
len(street_no)
street_no

['10-я Рабочая',
 '13-я Рабочая',
 '1-я Восточная',
 '1-я Встречная',
 '1-я Гражданская',
 '1-я Заречная',
 '1-я Крутая',
 '1-я Лесная',
 '1-я Линейная',
 '1-я Пригородная',
 '1-я Проходная',
 '1-я Северная',
 '1-я Центральная',
 '1-я Шоссейная',
 '20 лет ВЛКСМ',
 '2-я Водопроводная',
 '2-я Восточная',
 '2-я Гражданская',
 '2-я Крутая',
 '2-я Лесная',
 '2-я Пригородная',
 '2-я Центральная',
 '2-я Южная',
 '3-ий Ключ',
 '3-й Восточный',
 '3-я Восточная',
 '3-я Встречная',
 '3-я Гарнизонная',
 '3-я Круговая',
 '3-я Крутая',
 '3-я Лесная',
 '3-я Линейная',
 '3-я Матросская',
 '3-я Пригородная',
 '3-я Проходная',
 '3-я Северная',
 '3-я Центральная',
 '3-я Шоссейная',
 '3-я Южная',
 '4-ый Ключ',
 '4-я Круговая',
 '4-я Матросская',
 '4-я Пригородная',
 '4-я Террасная',
 '4-я Флотская',
 '4-я Центральная',
 '5-ый Ключ',
 '5-я Восточная',
 '5-я Проходная',
 '5-я Террасная',
 '5-я Центральная',
 '60 лет ВЛКСМ',
 '6-я Восточная',
 '6-я Матросская',
 '9-я Рабочая',
 'Абрикосовая',
 'Абрикосовый',

In [64]:
#Сохраняем улицы, которые с 0 объектов
writer = pd.ExcelWriter('База_ГИС_ЖКХ_улицы_no.xlsx')
pd.DataFrame(street_no).to_excel(writer, 'улицы_no')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_10060\540554558.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [74]:
#Ручной сбрщик по ВАРИАНТУ №2
soup = BeautifulSoup(driver.page_source, 'lxml')
t1 = []
t = []
a = 540  #необходимо постоянно менять на актуальное значение

#СРБИРАЕМ ТАБЛИЧКИ
table = soup.findAll('div', class_ = 'register-card__container_multi') #[2].findAll('td')[10].text.strip()
address = soup.findAll('div', class_ = 'register-card__header-title')  #адрес

for tb, ad in zip(table, address):
    t1.clear()
    t.clear()
    a = a + 1 #счетчик

    value = tb.findAll('td') #[17].text.strip()

    for vl in value:
        t.append(vl.text.strip())

    splits = np.array_split(t, len(value) / 2)
    for array in splits:
        t1.append(list(array))

    t1.append(['Адрес:', ad.text.strip()]) #добавляем адрес

    df = pd.DataFrame(t1)
    df = df.set_index(0)    

    if a == 1:
        df_result = df
    if a > 1:
        df_result = pd.concat([df_result, df], axis = 1)

df_result

,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0,,,,,,,,,,,,,,,,,,,,,
Кадастровый номер:,25:28:050059:565,25:28:050059:637,25:28:050059:1196,25:28:050023:814,25:28:050060:287,25:28:010022:299,25:28:010022:418,25:28:010024:313,25:28:010022:396,25:28:010024:1575,...,25:28:050004:634,25:28:050004:714,25:28:050004:1133,25:28:050004:1128,25:28:050002:2008,25:28:050002:1155,25:28:050002:1005,25:28:050077:1041,25:28:050002:1196,25:28:000000:23593
Год ввода в эксплуатацию:,1949,1904,-,1918,-,-,-,-,1950,-,...,-,-,-,-,-,-,-,-,1960,1964
Количество этажей:,1,1,2,1,2,3,1,1,1,2,...,1,1,3,3,1,1,1,1,1,1
Состояние:,-,-,Исправный,-,Исправный,Исправный,Исправный,Исправный,Исправный,Исправный,...,Исправный,-,Исправный,Исправный,Исправный,Исправный,Исправный,Исправный,-,-
Управляющая организация:,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
Адрес:,"690024, край Приморский, г Владивосток, ул Апу...","690024, край Приморский, г Владивосток, ул Апу...","690024, край Приморский, г Владивосток, ул Апу...","690024, край Приморский, г Владивосток, ул Апу...","690024, край Приморский, г Владивосток, ул Апу...","690001, край Приморский, г Владивосток, ул Сла...","690001, край Приморский, г Владивосток, ул Сла...","690001, край Приморский, г Владивосток, ул Сла...","690001, край Приморский, г Владивосток, ул Сла...","690001, край Приморский, г Владивосток, ул Сла...",...,"690912, край Приморский, г Владивосток, п Труд...","690912, край Приморский, г Владивосток, п Труд...","690912, край Приморский, г Владивосток, п Труд...","690912, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд...","690910, край Приморский, г Владивосток, п Труд..."
Дом находится всобственности субъектаРФ и…\n \n\n\n Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,NaN,NaN,Нет,NaN,Нет,Нет,Нет,Нет,Нет,Нет,...,Нет,NaN,Нет,Нет,NaN,NaN,NaN,Нет,NaN,NaN
Дом находится вмуниципальнойсобственности и…\n \n\n\n Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,NaN,NaN,Нет,NaN,Нет,Нет,Нет,Нет,Нет,Нет,...,Нет,Нет,Нет,Нет,Нет,Нет,Нет,Нет,NaN,Нет


In [75]:
a

540

In [76]:
df_result_copy = df_result.copy()                                   #делаем копию
df_result_copy = df_result_copy.transpose()                         #транспонируем, строки становятся столбцами

#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Жилое_доп.xlsx')
df_result_copy.to_excel(writer, 'База')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_6628\1564215196.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


# Проверка улиц на наличие объектов

In [ ]:
#ВАЖНО!
#Задаем Субъект РФ
#Задаем Город
#Задаем Тип дома
#Задаем Кадастровый номер - 25: (начало КН). Это позволит спарсить дома, по которым есть КН и скорее всего ГП (для Жилое)
#Для парсинга необходимо использовать ВАРИАНТ №2
#Необходимо обязательно выставить 100 объектов на странице
#Парсить надо только те улицы, по которым есть объекты. Перед парсингом необходимо собрать улицы с объектами
#Задержка по времени должна быть не менее 8 секунд, иначе кнопка Найти перестает работать.
#Чтобы спарсить все данные необходимо скачать всю базу с ГИС ЖКХ, отобрать улицы, сравнить их с нашими и допарсить те
#улицы, которые не прошли из-за дублирования, например, пер. 3-й Восточный и пр-кт. Восточный!!!
#Также, о. Русский, о. Попова, Трудовое - надо парсить отдельно во Владивостоке, так как это отдельные нп.

#Жилые, Владивосток, КН - 25:, общее количество 2679

In [109]:
#Подгружаем улицы
df_str = pd.read_excel('D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Квартиры_Владивосток\ГИС ЖКХ/База_ГИС_ЖКХ_streets.xlsx', sheet_name = 2)
df_str

,Unnamed: 0,Улицы_1,Улицы_2,Количество,Улицы_no
0,0,пр-кт. 100-летия Владивостока,100-летия Владивостока,121,NaN
1,1,ул. 10-я Рабочая,10-я Рабочая,0,10-я Рабочая
2,2,ул. 13-я Рабочая,13-я Рабочая,0,13-я Рабочая
3,3,ул. 1-я Восточная,1-я Восточная,0,1-я Восточная
4,4,ул. 1-я Встречная,1-я Встречная,0,1-я Встречная
...,...,...,...,...,...
996,996,ул. Ясеневая,Ясеневая,0,Ясеневая
997,997,ул. Ясная,Ясная,0,Ясная
998,998,пер. Ясный,Ясный,0,Ясный
999,999,ул. Яхтовая,Яхтовая,0,Яхтовая


In [127]:
street_no = [] #улицы которые не прошли и они пустые
street_big_100 = [] #улицы которые больше 100 домов
street_all_count = [] #количество домов по каждой улице

for stre in tqdm(range(0, len(df_str['Улицы_2']), 1), 'Сбор данных'):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    try:
        try:
            driver.find_element(By.XPATH, '//*[@id="s2id_streetpa1"]/a').click() #находим выпадающий список Улица
            input_3 = driver.find_element(By.XPATH, '//*[@id="s2id_autogen16_search"]') #находим input
            input_3.send_keys(df_str['Улицы_2'][stre]) #вносим в ячейку улицу
            sleep(2) #спим 2 секунды
            input_3.send_keys(Keys.ENTER) #жмем Enter
        except:
            #input_3.send_keys(Keys.ENTER) #жмем Enter
            winsound.Beep(2000, 3000) #громкий звук в 3 секунд
            sleep(30)       

        button = driver.find_element(By.XPATH, '/html/body/div[1]/div[8]/div/div[2]/houses-search-form/div[2]/div/ef-bp-form/div/form/div[3]/div/div[2]/button') #находим элемент кнопки Еще
        button.send_keys(Keys.ENTER) #нажимаем кнопку Найти
        sleep(6) #спим 6 секунд
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        N = int(soup.find('b', class_ = 'ng-binding').text) #количество объектов
        print(N)
        street_all_count.append(N)
        
        if N > 100:
            street_big_100.append(df_str['Улицы_2'][stre])

    except:
        #print(N)
        street_no.append(df_str['Улицы_2'][stre])                   #улицы которые не прошли, попадают в список

Сбор данных:   0%|                                                                  | 1/1001 [00:09<2:31:13,  9.07s/it]

0


Сбор данных:   0%|▏                                                                 | 2/1001 [00:17<2:25:35,  8.74s/it]

2


Сбор данных:   0%|▏                                                                 | 3/1001 [00:26<2:23:47,  8.64s/it]

...

1


Сбор данных: 100%|██████████████████████████████████████████████████████████████▉| 1000/1001 [2:23:58<00:08,  8.54s/it]

1


Сбор данных: 100%|███████████████████████████████████████████████████████████████| 1001/1001 [2:24:07<00:00,  8.64s/it]

0


In [130]:
len(street_all_count)

1001

In [129]:
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_count_ж.xlsx')
pd.DataFrame(street_all_count).to_excel(writer, 'count')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_10060\2415253060.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [131]:
street_no

[]

In [132]:
street_big_100

[]

# Прочее

In [421]:
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные_all.xlsx')
df_result.to_excel(writer, 'База')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\3522614385.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [447]:
soup = BeautifulSoup(driver.page_source, 'lxml')
N = int(soup.find('b', class_ = 'ng-binding').text)
N

0

In [402]:
#СОСТОЯНИЕ
soup = BeautifulSoup(driver.page_source, 'lxml')
sost = soup.findAll('td', class_ = 'col-xs-6')

sosts = []
sosts_2 = []

#for i, s in enumerate(sost):
for s in sost:
    try:
        sosts.append(s.find('span', class_ = 'ng-binding').text.strip())
    except:
        continue

i = 0
for s in sosts:
    if i % 3 == 0:
        sosts_2.append(s)
    i = i + 1
sosts_2

['Исправный',
 'Аварийный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный']

In [226]:
#Парсить по страницам не получится, так как ГИС дает увидеть и спарсить только первые 20 страниц 
a = 0
pages = 40

for p in tqdm(range(1, pages + 1, 1), 'Сбор данных'):
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    a = a + 1
    
    #АДРЕС
    address = driver.find_elements(By.CLASS_NAME, "register-card__header-title")
    #address[0].text

    #СОСТОЯНИЕ
    sost = soup.findAll('td', class_ = 'col-xs-6')

    sosts = []
    sosts_2 = []

    #for i, s in enumerate(sost):
    for s in sost:
        try:
            sosts.append(s.find('span', class_ = 'ng-binding').text.strip())
        except:
            continue

    i = 0
    for s in sosts:
        if i % 3 == 0:
            sosts_2.append(s)
        i = i + 1
    #sosts_2

    #КН, ГОД, МАТЕРИАЛ, ЭТАЖНОСТЬ + АДРЕС и СОСТОЯНИЕ
    table_2 = soup.findAll('td', class_ = 'col-xs-7')

    table_s_2 = []

    i = 0
    j = 0
    for tb in table_2:
        table_s_2.append(tb.text)
        i = i + 1
        if i % 4 == 0:
            j = j + 1
            #print(j, i, address[j - 1].text)
            table_s_2.append(address[j - 1].text)
            table_s_2.append(sosts_2[j - 1])
    #table_s_2
    
    #СОБИРАЕМ В ТАБЛИЦУ
    new_array = np.reshape(table_s_2, (100, 6))
    df = pd.DataFrame(new_array, columns = ['КН', 'Год', 'Материал', 'Этажность', 'Адрес', 'Cостояние'])

    if a == 1:
        df_result = df
    if a > 1:
        df_result = pd.concat([df_result, df], axis = 0)
    
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)") #прокрутка в самый низ
    sleep(1)
    button_2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[8]/div/houses-search-result/div/div[4]/div/div[1]/div/ul[3]/li[2]/a') #переключение страниц
    button_2.send_keys(Keys.ENTER) #жмем Enter
    sleep(5)


Сбор данных:  12%|████████▊                                                             | 5/40 [00:43<05:04,  8.70s/it]


ValueError: cannot reshape array of size 601 into shape (100,6)

# Ручник

In [358]:
soup = BeautifulSoup(driver.page_source, 'lxml')
a = a + 1

#АДРЕС
address = driver.find_elements(By.CLASS_NAME, "register-card__header-title")
#address[0].text

#СОСТОЯНИЕ
sost = soup.findAll('td', class_ = 'col-xs-6')

sosts = []
sosts_2 = []

#for i, s in enumerate(sost):
for s in sost:
    try:
        sosts.append(s.find('span', class_ = 'ng-binding').text.strip())
    except:
        continue

i = 0
for s in sosts:
    if i % 3 == 0:
        sosts_2.append(s)
    i = i + 1
#sosts_2

#КН, ГОД, МАТЕРИАЛ, ЭТАЖНОСТЬ + АДРЕС и СОСТОЯНИЕ
table_2 = soup.findAll('td', class_ = 'col-xs-7')

table_s_2 = []

i = 0
j = 0
for tb in table_2:
    table_s_2.append(tb.text)
    i = i + 1
    if i % 4 == 0:
        j = j + 1
        #print(j, i, address[j - 1].text)
        table_s_2.append(address[j - 1].text)
        table_s_2.append(sosts_2[j - 1])
#table_s_2

#СОБИРАЕМ В ТАБЛИЦУ
new_array = np.reshape(table_s_2, (100, 6))
df = pd.DataFrame(new_array, columns = ['КН', 'Год', 'Материал', 'Этажность', 'Адрес', 'Cостояние'])

In [359]:
df

,КН,Год,Материал,Этажность,Адрес,Cостояние
0,25:28:030014:317,1939,-,2,"690042, край Приморский, г Владивосток, ул Бас...",Исправный
1,25:28:030014:318,1930,не определен,2,"690042, край Приморский, г Владивосток, ул Бас...",Исправный
2,25:28:030014:4986,2021,индивидуальный проект,25,"690042, край Приморский, г Владивосток, ул Бас...",Исправный
3,25:28:030014:5274,2021,индивидуальный проект,25,"690042, край Приморский, г Владивосток, ул Бас...",Исправный
4,25:28:000000:8599,2012,-,8,"690042, край Приморский, г Владивосток, ул Бас...",Исправный
...,...,...,...,...,...,...
95,25:28:030018:16447,2008,83,11,"690080, край Приморский, г Владивосток, ул Сах...",Исправный
96,25:28:000000:12001,1961,отсут в тех.паспорт.,5,"690080, край Приморский, г Владивосток, ул Сах...",Исправный
97,25:28:030018:571,1966,1-515/5М,5,"690080, край Приморский, г Владивосток, ул Сах...",Исправный
98,25:28:030018:794,1958,не определен,2,"690080, край Приморский, г Владивосток, ул Сах...",Исправный


In [220]:
#Сохраняем в Excel
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные_20.xlsx')
df.to_excel(writer, 'marks')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\3213230758.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [228]:
table_s_2

['25:28:010042:160',
 '1985',
 'Панельный',
 '5',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 37',
 'Исправный',
 '25:28:010036:252',
 '1987',
 'Панельный',
 '9',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 38',
 'Исправный',
 '25:28:010042:201',
 '1985',
 'Панельный',
 '12',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 39',
 'Исправный',
 '25:28:010042:151',
 '1985',
 'Панельный',
 '5',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 41',
 'Исправный',
 '25:28:010042:182',
 '1994',
 'Панельный',
 '10',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 45',
 'Исправный',
 '25:28:010042:195',
 '1987',
 'Панельный',
 '9',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 47',
 'Исправный',
 '25:28:010042:191',
 '1987',
 'Панельный',
 '9',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 51',
 'Исправный',
 '25:28:010042:193',
 '1988',
 'Панельный',
 '9',
 '690109, край Приморский, г Владивосток, ул Нейбута, д. 53',
 'Ис

In [180]:
sost = soup.findAll('td', class_ = 'col-xs-6')

sosts = []
sosts_2 = []

#for i, s in enumerate(sost):
for s in sost:
    try:
        sosts.append(s.find('span', class_ = 'ng-binding').text.strip())
    except:
        continue

i = 0
for s in sosts:
    if i % 3 == 0:
        sosts_2.append(s)
    i = i + 1

sosts_2

['Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный',
 'Исправный']

In [182]:
table_1 = soup.findAll('td', class_ = 'col-xs-5')

table_s_1 = []

i = 0
j = 0
for tb in table_1:
    table_s_1.append(tb.text)
    i = i + 1
    if i % 4 == 0:
        #print(j, i)
        table_s_1.append('Адрес')
        table_s_1.append('Cостояние')
    
table_s_1

['Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Количество этажей:',
 'Адрес',
 'Cостояние',
 'Кадастровый номер:',
 'Год ввода в эксплуатацию:',
 'Серия/Тип проекта здания:',
 'Колич

In [184]:
table_2 = soup.findAll('td', class_ = 'col-xs-7')

table_s_2 = []

i = 0
j = 0
for tb in table_2:
    table_s_2.append(tb.text)
    i = i + 1
    if i % 4 == 0:
        j = j + 1
        #print(j, i, address[j - 1].text)
        table_s_2.append(address[j - 1].text)
        table_s_2.append(sosts_2[j - 1])
    
table_s_2

['25:28:010037:6026',
 '1985',
 'Панельный',
 '9',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 1',
 'Исправный',
 '25:28:010038:448',
 '1985',
 'отсутствует',
 '9',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 2',
 'Исправный',
 '25:28:010038:2564',
 '1985',
 'отсутствует',
 '12',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 4',
 'Исправный',
 '25:28:010037:7860',
 '2022',
 '-',
 '23',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 5',
 'Исправный',
 '25:28:010038:586',
 '1986',
 'отсутствует',
 '9',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 6',
 'Исправный',
 '25:28:010038:519',
 '1985',
 'отсуствует',
 '9',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 8',
 'Исправный',
 '25:28:010037:183',
 '1987',
 'Панельный',
 '12',
 '690013, край Приморский, г Владивосток, ул Невельского, д. 11',
 'Исправный',
 '25:28:010037:247',
 '1987',
 'Панельный',
 '12',
 '690013, край Приморский, г Владивосток

In [191]:
new_array = np.reshape(table_s_2, (10, 6))
new_array = pd.DataFrame(new_array, columns = ['КН', 'Год', 'Материал', 'Этажность', 'Адрес', 'Cостояние'])
new_array

,КН,Год,Материал,Этажность,Адрес,Cостояние
0,25:28:010037:6026,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
1,25:28:010038:448,1985,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
2,25:28:010038:2564,1985,отсутствует,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
3,25:28:010037:7860,2022,-,23,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
4,25:28:010038:586,1986,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
5,25:28:010038:519,1985,отсуствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
6,25:28:010037:183,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
7,25:28:010037:247,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
8,25:28:010037:6024,1986,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
9,25:28:010037:253,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный


In [193]:
df_result_1 = new_array.copy()
df_result_1

,КН,Год,Материал,Этажность,Адрес,Cостояние
0,25:28:010037:6026,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
1,25:28:010038:448,1985,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
2,25:28:010038:2564,1985,отсутствует,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
3,25:28:010037:7860,2022,-,23,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
4,25:28:010038:586,1986,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
5,25:28:010038:519,1985,отсуствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
6,25:28:010037:183,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
7,25:28:010037:247,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
8,25:28:010037:6024,1986,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
9,25:28:010037:253,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный


In [194]:
df_result = pd.concat([df_result_1, new_array], axis = 0)
df_result

,КН,Год,Материал,Этажность,Адрес,Cостояние
0,25:28:010037:6026,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
1,25:28:010038:448,1985,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
2,25:28:010038:2564,1985,отсутствует,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
3,25:28:010037:7860,2022,-,23,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
4,25:28:010038:586,1986,отсутствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
5,25:28:010038:519,1985,отсуствует,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
6,25:28:010037:183,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
7,25:28:010037:247,1987,Панельный,12,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
8,25:28:010037:6024,1986,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный
9,25:28:010037:253,1985,Панельный,9,"690013, край Приморский, г Владивосток, ул Нев...",Исправный


In [115]:
t1 = []

#Dictionary = dict(zip(table_s_1, table_s_2))
#resultList = list(map(list, Dictionary.items()))
# #t1.append(resultList)
for i in range(0, len(table_s_1), 1):
    t1.append([table_s_1[i], table_s_2[i]])

t1

[['Кадастровый номер:', '25:28:010037:6026'],
 ['Год ввода в эксплуатацию:', '1985'],
 ['Серия/Тип проекта здания:', 'Панельный'],
 ['Количество этажей:', '9'],
 ['Кадастровый номер:', '25:28:010038:448'],
 ['Год ввода в эксплуатацию:', '1985'],
 ['Серия/Тип проекта здания:', 'отсутствует'],
 ['Количество этажей:', '9'],
 ['Кадастровый номер:', '25:28:010038:2564'],
 ['Год ввода в эксплуатацию:', '1985'],
 ['Серия/Тип проекта здания:', 'отсутствует'],
 ['Количество этажей:', '12'],
 ['Кадастровый номер:', '25:28:010037:7860'],
 ['Год ввода в эксплуатацию:', '2022'],
 ['Серия/Тип проекта здания:', '-'],
 ['Количество этажей:', '23'],
 ['Кадастровый номер:', '25:28:010038:586'],
 ['Год ввода в эксплуатацию:', '1986'],
 ['Серия/Тип проекта здания:', 'отсутствует'],
 ['Количество этажей:', '9'],
 ['Кадастровый номер:', '25:28:010038:519'],
 ['Год ввода в эксплуатацию:', '1985'],
 ['Серия/Тип проекта здания:', 'отсуствует'],
 ['Количество этажей:', '9'],
 ['Кадастровый номер:', '25:28:01003

# Начиная с 20 страницы json отсутствует

In [355]:
def get_category(page = 1):
    #копируем cURL (bash), затем вставляем в https://curlconverter.com/, получем готовый код
    cookies = {
        'JSESSIONID': '"fmizSBrwMvlD_utSn6mogJ0R5ZBQlTQFIbjIAu9H.ppak-app-wf-ui-srv05:rest"',
        'route_pafo-saiku': '3eed4a51ebe9ecbd752731228dd302a5',
        'suimSessionGuid': 'f1353b90-f1be-469b-bf73-5e16305274e3',
        'route_rest': '9ab4234711eff9f497e7da0986c6cfd0',
        'route_suim': '9cec376355a82e1c2092d0f01142afd1',
        'route_pafo-reports': '3eed4a51ebe9ecbd752731228dd302a5',
    }

    headers = {
        #'Accept': 'application/json; charset=utf-8',
        'Accept': 'application/json, text/javascript, */*',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded',
        #'Content-Type': 'application/json;charset=UTF-8',
        #'Cookie': 'JSESSIONID="fmizSBrwMvlD_utSn6mogJ0R5ZBQlTQFIbjIAu9H.ppak-app-wf-ui-srv05:rest"; route_pafo-saiku=3eed4a51ebe9ecbd752731228dd302a5; suimSessionGuid=f1353b90-f1be-469b-bf73-5e16305274e3; route_rest=9ab4234711eff9f497e7da0986c6cfd0; route_suim=9cec376355a82e1c2092d0f01142afd1; route_pafo-reports=3eed4a51ebe9ecbd752731228dd302a5',
        'Origin': 'https://dom.gosuslugi.ru',
        'Referer': 'https://dom.gosuslugi.ru/',
        'Request-GUID': 'd4c27945-04b3-4ef3-9c1a-0c1c26b20df2',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Session-GUID': 'f1353b90-f1be-469b-bf73-5e16305274e3',
        'State-GUID': '/houses',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'pageIndex': str(page), #'pageIndex': '1'
        'elementsPerPage': '100',
    }

    json_data = {
        'regionCode': '43909681-d6e1-432d-b61f-ddac393cb5da',
        'cityCode': '7b6de6a5-86d0-4735-b11a-499081111af8',
        'fiasHouseCodeList': None,
        'estStatus': None,
        'strStatus': None,
        'calcCount': True,
        'houseConditionRefList': None,
        'houseTypeRefList': [
            '1',
        ],
        'houseManagementTypeRefList': None,
        'cadastreNumber': None,
        'oktmo': None,
        'statuses': [
            'APPROVED',
        ],
        'regionProperty': None,
        'municipalProperty': None,
        'hostelTypeCodes': None,
    }

    response = requests.post(
        'https://dom.gosuslugi.ru/homemanagement/api/rest/services/houses/public/searchByAddress',
        params = params,
        cookies = cookies,
        headers = headers,
        json = json_data,
    )
    return response.json()

def prepare_items(response):
    t1 = []
    
    t1_raw = response.get('items', None)
    
    if t1_raw != None and len(t1_raw) > 0:
        for t in t1_raw:
            t1.append({
                'Адрес': t.get('address', None).get('formattedAddress', None),
                'КН': t.get('cadastreNumber', None),
                'Год': t.get('buildingYear', None),
                'Материал': t.get('planSeries', None),
                'Этажность': t.get('maxFloorCount', None),
                'Состояние': t.get('houseCondition', None).get('houseCondition', None) if t.get('houseCondition', 
                                                                                                None) != None else None,
            })
    
    return t1

def main():
    pages = 40 #количество страниц
    a = 0
    pages_no = []
    
    for p in tqdm(range(1, pages + 1, 1), 'Сбор данных'):
        #try:
        a = a + 1 #счетчик
        response = get_category(page = p)
        sleep(10)
        #response = response.json()
        t1 = prepare_items(response)
        #print(t1)
        df = pd.DataFrame(t1)
        #df = df.set_index(0)

        if a == 1:
            df_result = df
        if a > 1:
            df_result = pd.concat([df_result, df], axis = 0)

        #Сохраняем в Excel
        writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные.xlsx')
        df_result.to_excel(writer, 'marks')
        writer.save()
        #except:
            #print('Error')
            #print(resp.text)
            #pages_no.append(p)
    
    #return pages_no
    
    #pd.DataFrame(t1).to_csv('База_ГИС_ЖКС.csv', index = False)

if __name__ == '__main__':
    main()    

Сбор данных:   0%|                                                                              | 0/40 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\135679790.py:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
Сбор данных:   2%|█▊                                                                    | 1/40 [00:10<06:34, 10.12s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\135679790.py:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
Сбор данных:   5%|███▌                                                                  | 2/40 [00:20<06:40, 10.54s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\135679790.py:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
Сбор данных:   8%|█

JSONDecodeError: [Errno Expecting value] : 0

In [86]:
pd.read_csv('База_ГИС_ЖКС.csv')

,Адрес,КН,Год,Материал,Этажность,Состояние
0,"690013, край Приморский, г Владивосток, ул Нев...",25:28:010037:6026,1985.0,Панельный,9,Исправный
1,"690013, край Приморский, г Владивосток, ул Нев...",25:28:010038:448,1985.0,отсутствует,9,Исправный
2,"690013, край Приморский, г Владивосток, ул Нев...",25:28:010038:2564,1985.0,отсутствует,12,Исправный
3,"690013, край Приморский, г Владивосток, ул Нев...",25:28:010037:7860,2022.0,NaN,23,Исправный
4,"690013, край Приморский, г Владивосток, ул Нев...",25:28:010038:586,1986.0,отсутствует,9,Исправный
...,...,...,...,...,...,...
95,"690012, край Приморский, г Владивосток, ул Кал...",25:28:030001:257,1956.0,отсут в тех.паспорт.,2,Исправный
96,"690012, край Приморский, г Владивосток, ул Кал...",25:28:030002:407,1948.0,"нет данных, Блочный",3,Исправный
97,"690012, край Приморский, г Владивосток, ул Кал...",25:28:030001:284,1957.0,нет данных,2,Исправный
98,"690012, край Приморский, г Владивосток, ул Кал...",25:28:030001:288,1956.0,NaN,2,Исправный


# Ручник

In [356]:
def get_category():
    cookies = {
        'JSESSIONID': '"fmizSBrwMvlD_utSn6mogJ0R5ZBQlTQFIbjIAu9H.ppak-app-wf-ui-srv05:rest"',
        'route_pafo-saiku': '3eed4a51ebe9ecbd752731228dd302a5',
        'suimSessionGuid': 'f1353b90-f1be-469b-bf73-5e16305274e3',
        'route_rest': '9ab4234711eff9f497e7da0986c6cfd0',
        'route_suim': '9cec376355a82e1c2092d0f01142afd1',
        'route_pafo-reports': '3eed4a51ebe9ecbd752731228dd302a5',
    }

    headers = {
        'Accept': 'application/json; charset=utf-8',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json;charset=UTF-8',
        # 'Cookie': 'JSESSIONID="JhzmATbSGgHve7Hgw4i5i7uqP9quGMUXKxScz87D.ppak-app-wf-ui-srv05:rest"; route_pafo-saiku=3eed4a51ebe9ecbd752731228dd302a5; suimSessionGuid=f1353b90-f1be-469b-bf73-5e16305274e3; route_rest=9ab4234711eff9f497e7da0986c6cfd0; route_suim=9cec376355a82e1c2092d0f01142afd1; route_pafo-reports=3eed4a51ebe9ecbd752731228dd302a5',
        'Origin': 'https://dom.gosuslugi.ru',
        'Referer': 'https://dom.gosuslugi.ru/',
        'Request-GUID': 'd4c27945-04b3-4ef3-9c1a-0c1c26b20df2',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Session-GUID': 'f1353b90-f1be-469b-bf73-5e16305274e3',
        'State-GUID': '/houses',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'pageIndex': '20', #'pageIndex': '1'
        'elementsPerPage': '100',
    }

    json_data = {
        'regionCode': '43909681-d6e1-432d-b61f-ddac393cb5da',
        'cityCode': '7b6de6a5-86d0-4735-b11a-499081111af8',
        'fiasHouseCodeList': None,
        'estStatus': None,
        'strStatus': None,
        'calcCount': True,
        'houseConditionRefList': None,
        'houseTypeRefList': [
            '1',
        ],
        'houseManagementTypeRefList': None,
        'cadastreNumber': None,
        'oktmo': None,
        'statuses': [
            'APPROVED',
        ],
        'regionProperty': None,
        'municipalProperty': None,
        'hostelTypeCodes': None,
    }

    response = requests.post(
        'https://dom.gosuslugi.ru/homemanagement/api/rest/services/houses/public/searchByAddress',
        params = params,
        cookies = cookies,
        headers = headers,
        json = json_data,
    )
    return response.json()

In [357]:
response = get_category()
t1_raw = response.get('items', None)

JSONDecodeError: [Errno Expecting value] : 0

In [348]:
t1_raw

[{'guid': '4fb55499-84dd-47d5-8dbf-5cb1ad0a9dbf',
  'rootGuid': None,
  'versionNumber': None,
  'lastUpdateUnixTime': None,
  'lastUpdateDate': None,
  'createDate': None,
  'readOnly': None,
  'active': None,
  'asyncProcessing': None,
  'asyncProcessType': None,
  'status': 'APPROVED',
  'cancellationReasonCode': None,
  'cancellationComment': None,
  'cancellationDate': None,
  'demolitionDate': None,
  'demolitionReason': None,
  'address': {'region': {'guid': '00170cae-8b4e-47ad-8473-27987a9644d7',
    'actual': True,
    'code': None,
    'rootEntityGuid': None,
    'lastUpdateDate': None,
    'createDate': None,
    'aoGuid': '43909681-d6e1-432d-b61f-ddac393cb5da',
    'aoLevel': 1,
    'postalCode': None,
    'formalName': 'Приморский',
    'offName': 'Приморский',
    'shortName': 'край',
    'parentGuid': None,
    'oktmo': None,
    'regionCode': '25',
    'autoCode': None,
    'areaCode': None,
    'cityCode': None,
    'ctarCode': None,
    'placeCode': None,
    'planCod

In [349]:
t1 = []

#t1_raw = response.get('items', None)

if t1_raw != None and len(t1_raw) > 0:
    for t in t1_raw:
        t1.append({
            'Адрес': t.get('address', None).get('formattedAddress', None),
            'КН': t.get('cadastreNumber', None),
            'Год': t.get('buildingYear', None),
            'Материал': t.get('planSeries', None),
            'Этажность': t.get('maxFloorCount', None),
            'Состояние': t.get('houseCondition', None).get('houseCondition', None) if t.get('houseCondition', 
                                                                                            None) != None else None,
        })
t1

[{'Адрес': '690042, край Приморский, г Владивосток, ул Басаргина, д. 2',
  'КН': '25:28:030014:317',
  'Год': None,
  'Материал': None,
  'Этажность': '2',
  'Состояние': 'Исправный'},
 {'Адрес': '690042, край Приморский, г Владивосток, ул Басаргина, д. 4',
  'КН': '25:28:030014:318',
  'Год': '1930',
  'Материал': 'не определен',
  'Этажность': '2',
  'Состояние': 'Исправный'},
 {'Адрес': '690042, край Приморский, г Владивосток, ул Басаргина, д. 6Б, к. 2',
  'КН': '25:28:030014:4986',
  'Год': '2021',
  'Материал': 'индивидуальный проект',
  'Этажность': '25',
  'Состояние': 'Исправный'},
 {'Адрес': '690042, край Приморский, г Владивосток, ул Басаргина, д. 6Б, к. 1',
  'КН': '25:28:030014:5274',
  'Год': '2021',
  'Материал': 'индивидуальный проект',
  'Этажность': '25',
  'Состояние': 'Исправный'},
 {'Адрес': '690042, край Приморский, г Владивосток, ул Басаргина, д. 15',
  'КН': '25:28:000000:8599',
  'Год': '2012',
  'Материал': None,
  'Этажность': '8',
  'Состояние': 'Исправный'},

In [350]:
df = pd.DataFrame(t1)
df

,Адрес,КН,Год,Материал,Этажность,Состояние
0,"690042, край Приморский, г Владивосток, ул Бас...",25:28:030014:317,None,None,2,Исправный
1,"690042, край Приморский, г Владивосток, ул Бас...",25:28:030014:318,1930,не определен,2,Исправный
2,"690042, край Приморский, г Владивосток, ул Бас...",25:28:030014:4986,2021,индивидуальный проект,25,Исправный
3,"690042, край Приморский, г Владивосток, ул Бас...",25:28:030014:5274,2021,индивидуальный проект,25,Исправный
4,"690042, край Приморский, г Владивосток, ул Бас...",25:28:000000:8599,2012,None,8,Исправный
...,...,...,...,...,...,...
95,"690080, край Приморский, г Владивосток, ул Сах...",25:28:030018:16447,2008,83,11,Исправный
96,"690080, край Приморский, г Владивосток, ул Сах...",25:28:000000:12001,1961,отсут в тех.паспорт.,5,Исправный
97,"690080, край Приморский, г Владивосток, ул Сах...",25:28:030018:571,1966,1-515/5М,5,Исправный
98,"690080, край Приморский, г Владивосток, ул Сах...",25:28:030018:794,1958,не определен,2,Исправный


In [351]:
writer = pd.ExcelWriter('База_ГИС_ЖКХ_Многоквартирные_20.xlsx')
df.to_excel(writer, 'marks')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_2948\4012922014.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [10]:
url = 'https://dom.gosuslugi.ru/#!/houses'

r = requests.get(url)
r.text

'<!DOCTYPE html>\n<!-- ng-strict-di="true" -->\n<html ng-controller="PublicCabinetController" ng-csp="no-unsafe-eval">\n    <head>\n        <script type="text/javascript">\n            ;(function () {\n                var loc = window.location.href\n                if (loc.indexOf(\'/#/\') > -1 && loc.indexOf(\'#!\') === -1) {\n                    window.location.href = loc.replace(\'/#/\', \'/#!/\')\n                }\n            })()\n        </script>\n        <title ng-bind="pageTitleHeader">Государственная информационная система жилищно-коммунального хозяйств</title>\n\n        <link rel="icon" type="image/x-icon" href="assets/img/favicon.ico" />\n        <link rel="shortcut icon" type="image/x-icon" href="assets/img/favicon.ico" />\n        <link type="image/x-icon" href="assets/img/favicon.ico" />\n\n        <meta charset="utf-8" />\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="viewport" content="width=device-width, initial-

In [11]:
soup = BeautifulSoup(r.text, 'lxml')
soup

<!DOCTYPE html>
<!-- ng-strict-di="true" --><html ng-controller="PublicCabinetController" ng-csp="no-unsafe-eval">
<head>
<script type="text/javascript">
            ;(function () {
                var loc = window.location.href
                if (loc.indexOf('/#/') > -1 && loc.indexOf('#!') === -1) {
                    window.location.href = loc.replace('/#/', '/#!/')
                }
            })()
        </script>
<title ng-bind="pageTitleHeader">Государственная информационная система жилищно-коммунального хозяйств</title>
<link href="assets/img/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="assets/img/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="assets/img/favicon.ico" type="image/x-icon"/>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="e1d1ef783c8c1e67" name="

In [12]:
soup.find('div', class_ = 'register-card__header-title')